In [69]:
import pandas as pd
import torch
from transformers import AutoModelForSeq2SeqLM, T5TokenizerFast


###  My initial idea was fine-tune some T5 model, but in one noght my computer did not managed to make even one epoch
```ModelFineTune.py``` is that attempt

So now my plan is:

### Plan:

1. Find T5 model 
2. Generate the summary for test texts with just pretrained model


As for text preprocessing:

1. remove any punctuation, numbers and non-russian text

In [70]:
DATA_PATH_TEST = './Data/test.csv'

MODEL_NAME = 'UrukHan/t5-russian-summarization'
MAX_INPUT = 256

### 1. Proprocess texts

In [71]:
import string
import re
import string

# remove punctuation
def remove_punct(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.replace('\n', ' ')
    text = text.translate(str.maketrans('', '', string.digits))
    return text

# keep only russian words
def remove_non_russian_words(text):
    russian_pattern = re.compile(r'[А-Яа-я]+')
    russian_words = russian_pattern.findall(text)
    cleaned_text = ' '.join(russian_words)
    return cleaned_text

In [76]:
test_df = pd.read_csv(DATA_PATH_TEST)
print(test_df)
test_df = test_df.loc[test_df.site_text.notna()]
# Apply the remove_punct function to 'site_text'
test_df['site_text_cleaned'] = test_df['site_text'].apply(remove_punct)
# Apply the remove_non_russian_words function to 'site_text'
test_df['site_text_cleaned'] = test_df['site_text'].apply(remove_non_russian_words)
print(test_df['site_text_cleaned'])

        id                                          site_text
0        0  HANNAH MARTIN\nОткрой для себя \nновый стиль с...
1        1  Преимущества\nКак работает?\nУстановка\nЗаказа...
2        2  +7 (812) 916 49 58\nПН-ПТ с 10 до 20 \nНаш Tel...
3        3  Previous\nСовершай платежи \n в любое время, в...
4        4  Остекление балконов\nАлюминиевое остекление\nП...
...    ...                                                ...
1995  1995  Рассчитать стоимость фундамента\nСтроения\nКал...
1996  1996   Канализация для загородного дома «под ключ»\n...
1997  1997  +7 495 974-79-79\npartners@itpss.ru\n5 провере...
1998  1998  +7 (499) 499-46-96\nНабор веса за 4 недели Био...
1999  1999  Современный сервис \n для управления инвестици...

[2000 rows x 2 columns]
0       Открой для себя новый стиль с Начать покупки Ч...
1       Преимущества Как работает Установка Заказать У...
2       ПН ПТ с до Наш чатик ТУТ Добро пожаловать Вход...
3       Совершай платежи в любое время в любой точке К...

### 2.Generate banners

In [ ]:
tokenizer = T5TokenizerFast.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

input_sequences = test_df['site_text_cleaned'].values.tolist()
task_prefix = "Spell correct:"
decoded_outputs = []
i = 0

for sequence in input_sequences:
    i += 1
    if i % 10 == 0:
        print(i)

    encoded = tokenizer(sequence, padding="longest", max_length=MAX_INPUT, truncation=True, return_tensors="pt")
    encoded.to(device)

    generated_ids = model.generate(
        input_ids=encoded['input_ids'],
        attention_mask=encoded['attention_mask'],
        max_length=15,
        num_return_sequences=1
    )

    decoded_output = tokenizer.batch_decode(generated_ids.to("cpu"), skip_special_tokens=True)
    decoded_outputs.append(decoded_output)

decoded_outputs = [output.to("cpu") for output in decoded_outputs]
test_df['decoded_outputs'] = decoded_outputs

- So now my df has generated banners, in a column "decoded outputs"

### 3. Getting embeddings and making submission file

In [ ]:
from sentence_transformers import SentenceTransformer

test_df = pd.read_csv('test_with_generated.csv').drop(columns=['Unnamed: 0', 'site_text', 'site_text_cleaned'])

# here we had empty site text, so I just made it uo
new_row_data = {
    'id': '988',
    'decoded_outputs': 'Всем привет!',
}

new_row_df = pd.DataFrame([new_row_data])

test_df = pd.concat([test_df.iloc[:988], new_row_df, test_df.iloc[988:]], ignore_index=True)

In [ ]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

embeddings = model.encode(test_df['decoded_outputs'].tolist())
flat_embeddings = embeddings.flatten()

embedding_df = pd.DataFrame({'id': test_df['id'].repeat(384), 'val': flat_embeddings})
embedding_df['siteId_eId'] = [f"{siteId}_{eId}" for siteId in range(len(test_df)) for eId in range(384)]
embedding_df = embedding_df[['id', 'siteId_eId', 'val']]

# Save the DataFrame to a CSV file
embedding_df.to_csv('banner_embeddings.csv', index=False)